In [1]:
import geopandas as gpd
from ipywidgets import AppLayout, Dropdown, HBox, interactive_output, Layout, ToggleButton, VBox
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from shapely.geometry import Point

%matplotlib widget
plt.ioff()

In [ ]:
#test_tree = np.array([random.randint(0, 1) for i in range(30)]).reshape(1, -1)

In [2]:
# Load encoded data.
df_enc = pd.read_csv('./data/data_preprocessed/features.csv.gz', compression='gzip')
targetvars = df_enc[df_enc.columns[-3:]]

# Generate tree coordinates.
df = pd.read_csv('./data/data_raw/nyc_trees/nyc_tree_census_2015.csv.gz', compression='gzip')
nyc_trees = gpd.GeoDataFrame(df, geometry=[Point(coordinates) for coordinates in zip(df['longitude'], df['latitude'])])
nyc_trees.crs = {'init': 'epsg:4326', 'no_defs': True}

# New York boroughs.
nyc_boroughs = gpd.read_file('./data/data_raw/nyc_boroughs/nyc_geo.shp')
nyc_boroughs = nyc_boroughs.to_crs(epsg=4326)
nyc_boroughs.crs = {'init': 'epsg:4326', 'no_defs': True}

# New York streets.
nyc_streets = gpd.read_file('./data/data_raw/nyc_streets/geo_export_f311eef5-5969-43eb-a781-39fa54d09b70.shp')

# Load classifier.
model = joblib.load('gridsearch_lgbm_clf.pkl')

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.22.2.post1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.22.2.post1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# Tree Map: Street trees in New York

In [ ]:
# Specifiy dropdown menu for displaying trees.
sel_boroughs = Dropdown(options=[('All street trees', 1),
                                 ('Healthy street trees', 2),
                                 ('Dead street trees', 3),
                                 ('Ok street trees', 4)],
                        value=1,
                        description='Display:')
sel_boroughs.layout.margin = '0px 30% 0px 30%'
sel_boroughs.layout.width = '50%'
sel_boroughs.layout.padding = '2.0'

fig, ax = plt.subplots(edgecolor='black',
                       figsize=(13, 4))
fig.canvas.header_visible = False
fig.canvas.layout.height = '100%'
fig.canvas.layout.width = '100%'
ax.axis('off')

# Plot NYC boroughs, streets, and trees.
nyc_boroughs.plot(alpha=0.7, ax=ax, color='grey', edgecolor='k')
for borough in nyc_boroughs.itertuples(index=True, name='bname'):
    plt.text(borough.geometry.centroid.x,
             borough.geometry.centroid.y,
             borough.bname, 
             bbox=dict(boxstyle='square, pad=0.3',
                       edgecolor='k',
                       facecolor='white'))

# Plot streets for additional context.
nyc_streets.plot(ax=ax, alpha=0.2, color='grey')
tree_visual = nyc_trees.plot(ax=ax, alpha=1, color='white', edgecolor='white', markersize=1.4)

def update_view(change, tree_visual=tree_visual):
    """
    Updates the figure drawn above.
    """
    
    if change.new == 2:
        tree_visual = nyc_trees.loc[df_enc.loc[df_enc['health_Good']==1].index]
        tree_visual.plot(ax=ax, color='green', edgecolor='green', markersize=1.4)
    elif change.new == 3:
        tree_visual = nyc_trees.loc[df_enc.loc[df_enc['health_Dead|Stump']==1].index]
        tree_visual.plot(ax=ax, color='red', edgecolor='red', markersize=1.4)
    elif change.new == 4:
        tree_visual = nyc_trees.loc[df_enc.loc[df_enc['health_Poor|Fair']==1].index]
        tree_visual.plot(ax=ax, color='orange', edgecolor='orange', markersize=1.4)
    else:
        tree_visual = nyc_trees.plot(ax=ax, alpha=1, color='white', edgecolor='white', markersize=1.4)
        fig.canvas.draw()
    
    return tree_visual
    
sel_boroughs.observe(update_view, names='value')

AppLayout(header=None,
          left_sidebar=None,
          center=fig.canvas,
          right_sidebar=None,
          footer=sel_boroughs,
          pane_heights=[0, 13, 1],
          grid_gap='2px')

# Street Tree Questionnaire

In [3]:
model

{'health_Dead|Stump': {'classifier': GridSearchCV(cv=2, error_score=0,
               estimator=LGBMClassifier(boosting_type='dart', class_weight=None,
                                        colsample_bytree=1.0,
                                        importance_type='split',
                                        learning_rate=0.1, max_depth=-1,
                                        min_child_samples=20,
                                        min_child_weight=0.001,
                                        min_split_gain=0.0, n_estimators=100,
                                        n_jobs=-1, num_leaves=31,
                                        objective='binary', random_state=34,
                                        reg_alpha=0.0, reg_lambda=0.0,
                                        silent=False, subsample=1.0,
                                        subsample_for_bin=200000,
                                        subsample_freq=0),
               iid='deprecated', n_j

In [4]:
def classify_tree(tree, classifiers, targets):
    # tree = np.array([[1,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0]])
    # classifiers = xgb_clfs
    # targets = y

    results = classifiers[list(classifiers.keys())[0]]['grid_classifier'].predict(tree)

    if results.sum() == 0 | results.sum() > 1:
        print("############################  OUTPUT  #############################")
        print("Given the current answers, this tree cannot be classified correctly.")
        print("###################################################################")
    else:
        if results[0][0] == 1:
            print("#########  OUTPUT  #########")
            print("This tree is dead or a stump.")
            print("############################")
        elif results[0][1] == 1:
            print("#####  OUTPUT  #####")
            print("This tree is healthy.")
            print("####################")
        else:
            print("#############  OUTPUT  #################")
            print("This tree is in a poor or fair condition.")
            print("########################################")
            
    return results


def create_tree_quest(a, b, c, d, e, f, g, h, i, j, k, l, m, classifiers=model, targets=targetvars):
    """
    Create a questionnaire for new trees.
    """
    
    # If there is a steward, in which condition is the tree's steward?
    if a == 1:
        template['steward_Alive'] = 1
        template['steward_Dead|Stump'] = 0
    elif a == 2:
        template['steward_Alive'] = 0
        template['steward_Dead|Stump'] = 1
    else:
        template['steward_Alive'] = np.nan
        template['steward_Dead|Stump'] = np.nan
    
    # If there is a guard, in which condition is the tree's guard?
    if b == 1:
        template['guards_Dead|Stump'] = 1
        template['guards_Harmful'] = 0
        template['guards_Helpful'] = 0
        template['guards_None'] = 0
        template['guards_Unsure'] = 0
    elif b == 2:
        template['guards_Dead|Stump'] = 0
        template['guards_Harmful'] = 1
        template['guards_Helpful'] = 0
        template['guards_None'] = 0
        template['guards_Unsure'] = 0
    elif b == 3:
        template['guards_Dead|Stump'] = 0
        template['guards_Harmful'] = 0
        template['guards_Helpful'] = 1
        template['guards_None'] = 0
        template['guards_Unsure'] = 0
    elif b == 4:
        template['guards_Dead|Stump'] = 0
        template['guards_Harmful'] = 0
        template['guards_Helpful'] = 0
        template['guards_None'] = 1
        template['guards_Unsure'] = 0
    elif b == 5:
        template['guards_Dead|Stump'] = 0
        template['guards_Harmful'] = 0
        template['guards_Helpful'] = 0
        template['guards_None'] = 0
        template['guards_Unsure'] = 1
    else:
        template['guards_Dead|Stump'] = np.nan
        template['guards_Harmful'] = np.nan
        template['guards_Helpful'] = np.nan
        template['guards_None'] = np.nan
        template['guards_Unsure'] = np.nan
        
    # In which condition is the sidewalk next to the tree?
    if c == 1:
        template['sidewalk_Damage'] = 1
        template['sidewalk_Dead|Stump'] = 0
        template['sidewalk_NoDamage'] = 0
    elif c == 2:
        template['sidewalk_Damage'] = 0
        template['sidewalk_Dead|Stump'] = 1
        template['sidewalk_NoDamage'] = 0
    elif c == 3:
        template['sidewalk_Damage'] = 0
        template['sidewalk_Dead|Stump'] = 0
        template['sidewalk_NoDamage'] = 1
    else:
        template['sidewalk_Damage'] = np.nan
        template['sidewalk_Dead|Stump'] = np.nan
        template['sidewalk_NoDamage'] = np.nan
        
    # Are there stones at the root of the tree?
    if d == 1:
        template['root_stone_No'] = 0
        template['root_stone_Yes'] = 1
    elif d == 2:
        template['root_stone_No'] = 1
        template['root_stone_Yes'] = 0
    else:
        template['root_stone_No'] = np.nan
        template['root_stone_Yes'] = np.nan    
        
    # Is there a grate at the root of the tree?
    if e == 1:
        template['root_grate_No'] = 0
        template['root_grate_Yes'] = 1
    elif e == 2:
        template['root_grate_No'] = 1
        template['root_grate_Yes'] = 0
    else:
        template['root_grate_No'] = np.nan
        template['root_grate_Yes'] = np.nan
        
    # Is there something else at the root of the tree?
    if f == 1:
        template['root_other_No'] = 0
        template['root_other_Yes'] = 1
    elif f == 2:
        template['root_other_No'] = 1
        template['root_other_Yes'] = 0
    else:
        template['root_other_No'] = np.nan
        template['root_other_Yes'] = np.nan
        
    # Is there a wire at the trunk of the tree?
    if g == 1:
        template['trunk_wire_No'] = 0
        template['trunk_wire_Yes'] = 1
    elif g == 2:
        template['trunk_wire_No'] = 1
        template['trunk_wire_Yes'] = 0
    else:
        template['trunk_wire_No'] = np.nan
        template['trunk_wire_Yes'] = np.nan
        
    # Is there a light installation at the trunk of the tree?
    if h == 1:
        template['trnk_light_No'] = 0
        template['trnk_light_Yes'] = 1
    elif h == 2:
        template['trnk_light_No'] = 1
        template['trnk_light_Yes'] = 0
    else:
        template['trnk_light_No'] = np.nan
        template['trnk_light_Yes'] = np.nan
        
    # Is there something else at the trunk of the tree?
    if i == 1:
        template['trnk_other_No'] = 0
        template['trnk_other_Yes'] = 1
    elif i == 2:
        template['trnk_other_No'] = 1
        template['trnk_other_Yes'] = 0
    else:
        template['trnk_other_No'] = np.nan
        template['trnk_other_Yes'] = np.nan

    # Are the branches affected by any street light installations?
    if j == 1:
        template['brch_light_No'] = 0
        template['brch_light_Yes'] = 1
    elif j == 2:
        template['brch_light_No'] = 1
        template['brch_light_Yes'] = 0
    else:
        template['brch_light_No'] = np.nan
        template['brch_light_Yes'] = np.nan
        
    # Are the branches affected by any shoes?
    if k == 1:
        template['brch_shoe_No'] = 0
        template['brch_shoe_Yes'] = 1
    elif k == 2:
        template['brch_shoe_No'] = 1
        template['brch_shoe_Yes'] = 0
    else:
        template['brch_shoe_No'] = np.nan
        template['brch_shoe_Yes'] = np.nan
        
    # Are the branches affected by anything else?
    if l == 1:
        template['brch_other_No'] = 0
        template['brch_other_Yes'] = 1
    elif l == 2:
        template['brch_other_No'] = 1
        template['brch_other_Yes'] = 0
    else:
        template['brch_other_No'] = np.nan
        template['brch_other_Yes'] = np.nan
        
    # How many neighboring street trees are there within 10m from a tree?
    if m == 1:
        template['n_neighbors_no neighbor'] = 1
        template['n_neighbors_one neighbor'] = 0
        template['n_neighbors_two or more neighbors'] = 0
    elif m == 2:
        template['n_neighbors_no neighbor'] = 0
        template['n_neighbors_one neighbor'] = 1
        template['n_neighbors_two or more neighbors'] = 0
    elif m == 3:
        template['n_neighbors_no neighbor'] = 0
        template['n_neighbors_one neighbor'] = 0
        template['n_neighbors_two or more neighbors'] = 1
    else:
        template['n_neighbors_no neighbor'] = np.nan
        template['n_neighbors_one neighbor'] = np.nan
        template['n_neighbors_two or more neighbors'] = np.nan
        
    if template.isnull().any():
        print("#############  OUTPUT  #################")
        print("Please provide answers to all questions.")
        print("########################################")
    else:
        # Classify tree. Must provide a list of a Pandas series, otherwise returns a ValueError "feature_names mismatch"
        # when using XGBoost.
        query = np.array([template.astype('int64')])
        tree_health = classify_tree(query, classifiers, targets)
    
    return template


def reset_answers(args):
    """
    Reset the answers provided to the questionnaire.
    """
    
    a.value = 0
    b.value = 0
    c.value = 0
    d.value = 0
    e.value = 0
    f.value = 0
    g.value = 0
    h.value = 0
    i.value = 0
    j.value = 0
    k.value = 0
    l.value = 0
    m.value = 0
    
    
template = pd.Series(index=['steward_Alive',
                            'steward_Dead|Stump',
                            'guards_Dead|Stump',
                            'guards_Harmful',
                            'guards_Helpful',
                            'guards_None',
                            'guards_Unsure',
                            'sidewalk_Damage',
                            'sidewalk_Dead|Stump',
                            'sidewalk_NoDamage',
                            'root_stone_No',
                            'root_stone_Yes',
                            'root_grate_No',
                            'root_grate_Yes',
                            'root_other_No',
                            'root_other_Yes',
                            'trunk_wire_No',
                            'trunk_wire_Yes',
                            'trnk_light_No',
                            'trnk_light_Yes',
                            'trnk_other_No',
                            'trnk_other_Yes',
                            'brch_light_No',
                            'brch_light_Yes',
                            'brch_shoe_No',
                            'brch_shoe_Yes',
                            'brch_other_No',
                            'brch_other_Yes',
                            'n_neighbors_no neighbor',
                            'n_neighbors_one neighbor',
                            'n_neighbors_two or more neighbors'],
                    dtype='int64')
        
a = Dropdown(options=[('Please answer', 0),
                      ('Alive', 1),
                      ('No steward', 2)],
             description="If there is a steward, in which condition is the tree's steward?",
             layout=Layout(width='650px'),
             style={'description_width': '420px'},
             value=0)

b = Dropdown(options=[('Please answer', 0),
                      ("Don't see a tree", 1),
                      ('Harmful', 2),
                      ('Helpful', 3),
                      ('No guard', 4),
                      ('Unsure', 5)],
             description="If there is a guard, in which condition is the tree's guard?",
             layout=a.layout,
             style=a.style,
             value=0)

c = Dropdown(options=[('Please answer', 0),
                      ('Damage', 1),
                      ("Don't see a tree", 2),
                      ('No damage', 3)],
             description='In which condition is the sidewalk next to the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

d = Dropdown(options=[('Please answer', 0),
                      ('Stones at the root', 1),
                      ('No stones at the root', 2)],
             description='Are there stones at the root of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)
                      
e = Dropdown(options=[('Please answer', 0),
                      ('Grate at the root', 1),
                      ('No grate at the root', 2)],
             description='Is there a grate at the root of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

f = Dropdown(options=[('Please answer', 0),
                      ('Something else at the root', 1),
                      ('Nothing else at the root', 2)],
             description='Is there something else at the root of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

g = Dropdown(options=[('Please answer', 0),
                      ('Wire touches trunk', 1),
                      ('No wire at trunk', 2)],
             description='Is there a wire at the trunk of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

h = Dropdown(options=[('Please answer', 0),
                      ('Light at the trunk', 1),
                      ('No light at the trunk', 2)],
             description='Is there a light installation at the trunk of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

i = Dropdown(options=[('Please answer', 0),
                      ('Something else at the trunk', 1),
                      ('Nothing at the trunk', 2)],
             description='Is there something else at the trunk of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

j = Dropdown(options=[('Please answer', 0),
                      ('Light installations on branches', 1),
                      ('No light installations on branches', 2)],
             description='Are the branches affected by any street light installations?',
             layout=a.layout,
             style=a.style,
             value=0)

k = Dropdown(options=[('Please answer', 0),
                      ('Shoes on branches', 1),
                      ('No shoes on branches', 2)],
             description='Are the branches affected by any shoes?',
             layout=a.layout,
             style=a.style,
             value=0)

l = Dropdown(options=[('Please answer', 0),
                      ('Something else on branches', 1),
                      ('Nothing affects the branches', 2)],
             description='Are the branches affected by anything else?',
             layout=a.layout,
             style=a.style,
             value=0)

m = Dropdown(options=[('Please answer', 0),
                      ('No neighboring tree', 2),
                      ('One neighboring tree', 2),
                      ('More than one neighboring tree', 3)],
             description='How many neighboring street trees are there within a 10m distance?',
             layout=a.layout,
             style=a.style,
             value=0)

out = interactive_output(create_tree_quest, {'a': a,
                                             'b': b,
                                             'c': c,
                                             'd': d,
                                             'e': e,
                                             'f': f,
                                             'g': g,
                                             'h': h,
                                             'i': i,
                                             'j': j,
                                             'k': k,
                                             'l': l,
                                             'm': m})

questionnaire = HBox([VBox([a, b, c, d, e, f, g, h, i, j, k, l, m]),
                      out])

reset_button = ToggleButton(button_style='info',
                            description='Reset answers',
                            layout=Layout(width='150px',
                                          heigth='auto',
                                           margin='right'))

reset_button.observe(reset_answers, 'value')
questionnaire.observe(reset_button)
                
AppLayout(header=None,
          left_sidebar=None,
          center=questionnaire,
          right_sidebar=None,
          footer=reset_button,
          pane_heights=[0, 13, 1],
          grid_gap='20px')

AppLayout(children=(ToggleButton(value=False, button_style='info', description='Reset answers', layout=Layout(…